In [2]:
## importing packages
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import imblearn
from collections import Counter
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold

In [3]:
## import data
df_features = pd.read_csv("train_features.csv")
df_features = df_features.sort_values(by=['pid'])
df_labels = pd.read_csv("train_labels.csv")
df_labels = df_labels.sort_values(by=['pid'])
print(df_labels) # ATTENTION: TIME IS NOT SORTED ANYMORE
df_features.count()

         pid  LABEL_BaseExcess  LABEL_Fibrinogen  LABEL_AST  \
0          1               1.0               0.0        0.0   
6622       2               0.0               0.0        0.0   
15008      4               0.0               0.0        0.0   
16335      6               1.0               0.0        0.0   
17676      8               0.0               0.0        0.0   
...      ...               ...               ...        ...   
14402  31653               0.0               0.0        0.0   
14403  31654               0.0               0.0        0.0   
14404  31656               1.0               0.0        0.0   
14405  31657               0.0               0.0        0.0   
14406  31658               0.0               0.0        0.0   

       LABEL_Alkalinephos  LABEL_Bilirubin_total  LABEL_Lactate  \
0                     0.0                    0.0            1.0   
6622                  0.0                    0.0            0.0   
15008                 0.0                 

pid                 227940
Time                227940
Age                 227940
EtCO2                 9783
PTT                  10299
BUN                  20105
Lactate              10756
Temp                 81115
Hgb                  22295
HCO3                 12559
BaseExcess           19887
RRate               187785
Fibrinogen            2493
Phosphate            11590
WBC                  19083
Creatinine           17792
PaCO2                21043
AST                   5761
FiO2                 26602
Platelets            18035
SaO2                 13014
Glucose              47036
ABPm                195889
Magnesium            17523
Potassium            28393
ABPd                152418
Calcium              17830
Alkalinephos          5708
SpO2                195192
Bilirubin_direct       719
Chloride             13917
Hct                  27297
Heartrate           200128
Bilirubin_total       5326
TroponinI             3776
ABPs                191650
pH                   25046
d

In [4]:
## feature imputation and engineering
all_features = ["pid", "Time", "Age", "EtCO2", "PTT", "BUN", "Lactate", "Temp", "Hgb", "HCO3", "BaseExcess", "RRate", "Fibrinogen", "Phosphate", "WBC", "Creatinine", "PaCO2", "AST", "FiO2", "Platelets", "SaO2", "Glucose", "ABPm", "Magnesium", "Potassium", "ABPd", "Calcium", "Alkalinephos", "SpO2", "Bilirubin_direct", "Chloride", "Hct", "Heartrate", "Bilirubin_total", "TroponinI", "ABPs", "pH"]
#all_features = ["pid", "Time", "Age", "EtCO2", "PTT", "BUN", "Lactate", "Temp", "Hgb", "HCO3", "BaseExcess", "RRate", "Fibrinogen", "Phosphate", "WBC", "Creatinine", "PaCO2", "AST", "FiO2", "Platelets", "SaO2", "Glucose", #"ABPm", "Magnesium", "Potassium", "ABPd", "Calcium", "Alkalinephos", "SpO2", "Bilirubin_direct", "Chloride", "Hct", "Heartrate", "Bilirubin_total", "TroponinI", "ABPs", "pH"]

task2_label = "LABEL_Sepsis"
age_span = 3

X = df_features
y = df_labels[task2_label]
#print(y)
all_pids = list(set(X['pid']))

In [0]:

X_eng_dict = {"pid":[], "Age":[]}
for feat in all_features[3:]:
    X_eng_dict[f"{feat}_min"] = []
    X_eng_dict[f"{feat}_max"] = []
    X_eng_dict[f"{feat}_med"] = []
    X_eng_dict[f"{feat}_std"] = []
    #X_eng_dict[f"{feat}_a"] = []
    #X_eng_dict[f"{feat}_b"] = []

for pid in all_pids:
    X_pid = X[X["pid"] == pid].sort_values(by=['Time'])
    age = int(X_pid['Age'].iloc[0])
    medians_from_age = X[(X["Age"] <= age+age_span) & (X["Age"] >= age-age_span)].median()
    #print(medians_from_age)
    X_pid = X_pid.fillna(value=medians_from_age)
    X_eng_dict["Age"].append(age)
    X_eng_dict["pid"].append(pid)
    #print(X_pid)
    #pid_feats = np.array()
    for feat in all_features[3:]:
        feat_min = X_pid[feat].min()
        feat_min_name = f"{feat}_min"
        feat_max = X_pid[feat].max()
        feat_max_name = f"{feat}_max"
        feat_med = X_pid[feat].median()
        feat_med_name = f"{feat}_med"
        feat_std = X_pid[feat].std()
        feat_std_name = f"{feat}_std"
        X_eng_dict[feat_min_name].append(feat_min)
        X_eng_dict[feat_max_name].append(feat_max)
        X_eng_dict[feat_med_name].append(feat_med)
        X_eng_dict[feat_std_name].append(feat_std)
        #fit_x = X_pid["Time"]
        #fit_y = X_pid[feat]
        #(a,b) = np.polyfit(fit_x, fit_y, 1)
        #feat_a_name = f"{feat}_a"
        #feat_b_name = f"{feat}_b"
        #X_eng_dict[feat_a_name].append(a)
        #X_eng_dict[feat_b_name].append(b)
print('hey')
#X_eng_dict

In [9]:
X_eng = pd.DataFrame.from_dict(X_eng_dict)

NameError: name 'X_eng_dict' is not defined

In [4]:
# Create
#X_eng.to_pickle("X_eng_nofit.pkl")
#y = df_labels
[task2_label]
# Load
X_eng = pd.read_pickle("X_eng_nofit.pkl")

In [5]:
X_eng_nopid = X_eng.loc[:, X_eng.columns != 'pid']#.to_numpy()
X_eng_nopid
y_nopid = y#.to_numpy()#loc[:, y.columns != 'pid']
X_eng_nopid

Age  EtCO2_min  EtCO2_max  EtCO2_med  EtCO2_std  PTT_min  PTT_max  \
0       34       34.0       34.0       34.0        0.0     30.2     30.2   
1       86       30.0       30.0       30.0        0.0     31.8     33.8   
2       66       33.0       33.0       33.0        0.0     32.3     34.6   
3       66       33.0       33.0       33.0        0.0     32.3     53.8   
4       42       34.5       34.5       34.5        0.0     31.5     31.5   
...    ...        ...        ...        ...        ...      ...      ...   
18990   52       34.0       34.0       34.0        0.0     25.8     32.2   
18991   66       33.0       33.0       33.0        0.0     32.3     32.3   
18992   44       35.0       35.0       35.0        0.0     31.5     31.5   
18993   70       32.0       32.0       32.0        0.0     32.8     32.8   
18994   60       33.0       33.0       33.0        0.0     32.0     32.0   

       PTT_med       PTT_std  BUN_min  ...  TroponinI_med  TroponinI_std  \
0         30.2  7.421377e-15     12.0  ...          0.050   7.247438e-18   
1         33.8  5.773503e-01     23.0  ...          0.310   3.752777e-02   
2         32.3  6.639528e-01      8.0  ...          0.215   5.629165e-02   
3         32.3  6.206515e+00     17.0  ...          0.215   0.000000e+00   
4         31.5  0.000000e+00     13.0  ...          0.070   2.886751e-03   
...        ...           ...      ...  ...            ...            ...   
18990     32.2  1.847521e+00     11.0  ...          0.110   2.898975e-17   
18991     32.3  7.421377e-15     17.0  ...          0.215   0.000000e+00   
18992     31.5  0.000000e+00     14.0  ...          0.080   1.449488e-17   
18993     32.8  7.421377e-15     18.0  ...          0.160   2.898975e-17   
18994     32.0  0.000000e+00     13.0  ...          0.150   2.898975e-17   

       ABPs_min  ABPs_max  ABPs_med   ABPs_std  pH_min  pH_max  pH_med  \
0         100.0     142.0     111.0  12.086807    7.33    7.41    7.38   
1         120.0     148.0     128.0   8.750325    7.38    7.38    7.38   
2          86.0     168.0     121.5  29.626335    7.37    7.37    7.37   
3          94.0     110.0      99.0   5.773503    7.33    7.39    7.37   
4         120.0     239.0     206.5  35.724959    7.37    7.37    7.37   
...         ...       ...       ...        ...     ...     ...     ...   
18990      75.0     118.0     107.0  16.789246    7.33    7.37    7.37   
18991     119.0     187.0     160.0  17.028497    7.37    7.37    7.37   
18992      88.0     120.0     103.0  12.638026    7.25    7.37    7.37   
18993     101.0     124.0     111.0   7.279590    7.37    7.37    7.37   
18994     118.0     137.0     122.5   8.458455    7.36    7.36    7.36   

             pH_std  
0      2.314316e-02  
1      9.276721e-16  
2      9.276721e-16  
3      1.758098e-02  
4      9.276721e-16  
...             ...  
18990  1.556998e-02  
18991  9.276721e-16  
18992  4.097301e-02  
18993  9.276721e-16  
18994  0.000000e+00  

[18995 rows x 137 columns]

In [6]:
y_nopid

0        0.0
6622     0.0
15008    0.0
16335    0.0
17676    0.0
        ... 
14402    0.0
14403    0.0
14404    0.0
14405    0.0
14406    0.0
Name: LABEL_Sepsis, Length: 18995, dtype: float64

In [7]:
X_train_eval, X_test, y_train_eval, y_test = train_test_split(X_eng_nopid, y_nopid, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train_eval, y_train_eval, test_size=0.2)

In [8]:
used_feats = ['Age', 'EtCO2_min', 'EtCO2_max', 'EtCO2_med', 'EtCO2_std', 'PTT_min', 'PTT_max', 'PTT_med', 'PTT_std', 'BUN_min', 'BUN_max', 'BUN_med', 'BUN_std', 'Lactate_min', 'Lactate_max', 'Lactate_med', 'Lactate_std', 'Temp_min', 'Temp_max', 'Temp_med', 'Temp_std', 'Hgb_min', 'Hgb_max', 'Hgb_med', 'Hgb_std', 'HCO3_min', 'HCO3_max', 'HCO3_med', 'HCO3_std', 'BaseExcess_max', 'BaseExcess_med', 'BaseExcess_std', 'RRate_min', 'RRate_max', 'RRate_med', 'RRate_std', 'Fibrinogen_min', 'Fibrinogen_max', 'Fibrinogen_med', 'Phosphate_min', 'Phosphate_max', 'Phosphate_med', 'Phosphate_std', 'WBC_min', 'WBC_max', 'WBC_med', 'WBC_std', 'Creatinine_min', 'Creatinine_max', 'Creatinine_med', 'Creatinine_std', 'PaCO2_min', 'PaCO2_max', 'PaCO2_med', 'PaCO2_std', 'AST_min', 'AST_max', 'AST_med', 'FiO2_min', 'FiO2_max', 'FiO2_med', 'FiO2_std', 'Platelets_min', 'Platelets_max', 'Platelets_med', 'Platelets_std', 'SaO2_min', 'SaO2_max', 'SaO2_med', 'SaO2_std', 'Glucose_min', 'Glucose_max', 'Glucose_med', 'Glucose_std', 'ABPm_min', 'ABPm_max', 'ABPm_med', 'ABPm_std', 'Magnesium_min', 'Magnesium_max', 'Magnesium_med', 'Magnesium_std', 'Potassium_min', 'Potassium_max', 'Potassium_med', 'Potassium_std', 'ABPd_min', 'ABPd_max', 'ABPd_med', 'ABPd_std', 'Calcium_min', 'Calcium_max', 'Calcium_med', 'Calcium_std', 'Alkalinephos_min', 'Alkalinephos_max', 'Alkalinephos_med', 'Alkalinephos_std', 'SpO2_min', 'SpO2_max', 'SpO2_med', 'SpO2_std', 'Bilirubin_direct_min', 'Bilirubin_direct_max', 'Bilirubin_direct_med', 'Bilirubin_direct_std', 'Chloride_min', 'Chloride_max', 'Chloride_med', 'Chloride_std', 'Hct_min', 'Hct_max', 'Hct_med', 'Hct_std', 'Heartrate_min', 'Heartrate_max', 'Heartrate_med', 'Heartrate_std', 'Bilirubin_total_min', 'Bilirubin_total_max', 'Bilirubin_total_med', 'Bilirubin_total_std', 'TroponinI_min', 'TroponinI_max', 'TroponinI_med', 'TroponinI_std', 'ABPs_min', 'ABPs_max', 'ABPs_med', 'ABPs_std', 'pH_min', 'pH_max', 'pH_med', 'pH_std']
#not_feat = ["Calcium_med", "Magnesium_med","Potassium_med", "Alkalinephos_med", "Bilirubin_direct_max", "Bilirubin_total_med", "pH_med", "Lactate_med", "EtCO2_max", "BUN_med", "Magnesium_med", "SaO2_med"]
#not_feat = ["Magnesium_min"]
X_train2 = X_train
X_test2 = X_test
X_val2 = X_val


counter = Counter(y_train)
# estimate scale_pos_weight value
estimate = counter[0] / counter[1]
print('Estimate: %.3f' % estimate)

#for ft in not_feat:
#    X_train2 = X_train2.loc[:, X_train2.columns != ft]
#    X_test2 = X_test2.loc[:, X_test2.columns != ft]
#    X_val2 = X_val2.loc[:, X_val2.columns != ft]
scores = {}
#for ex in np.linspace(0.01,0.07, 20):
for i in [""]:#np.linspace(0.1,0.9,10):
    xgb = XGBClassifier(scale_pos_weight=0.042, objective = "binary:logistic", base_score=0.25, eta=0.244)
    xgb.fit(X_train2, y_train, early_stopping_rounds=10, eval_metric="auc", eval_set = [(X_val2, y_val)], verbose=True)
    y_pred_proba = xgb.predict_proba(X_test2)
    y_pred = y_pred_proba[:,1]
    sc = roc_auc_score(y_test, y_pred)
    scores[i] = sc
#print(sc, uf)
scores

Estimate: 16.541


[0]	validation_0-auc:0.50000


Will train until validation_0-auc hasn't improved in 10 rounds.


[1]	validation_0-auc:0.50000


[2]	validation_0-auc:0.50000


...

[34]	validation_0-auc:0.74344


[35]	validation_0-auc:0.74492


[36]	validation_0-auc:0.74255


Stopping. Best iteration:
[26]	validation_0-auc:0.74716



{'': 0.7307769612385935}

In [74]:
scores

{}

Estimate: 16.459


Counter({0.0: 17907, 1.0: 1088})

In [5]:
X_handin = pd.read_csv("test_features.csv")


In [6]:
all_pids_handin = list(set(X_handin['pid']))
X_handin_eng_dict = {"pid":[], "Age":[]}
for feat in all_features[3:]:
    X_handin_eng_dict[f"{feat}_min"] = []
    X_handin_eng_dict[f"{feat}_max"] = []
    X_handin_eng_dict[f"{feat}_med"] = []
    X_handin_eng_dict[f"{feat}_std"] = []
    X_handin_eng_dict[f"{feat}_a"] = []
    X_handin_eng_dict[f"{feat}_b"] = []

for pid in all_pids_handin:
    X_pid = X_handin[X_handin["pid"] == pid].sort_values(by=['Time'])
    age = int(X_pid['Age'].iloc[0])
    medians_from_age = X_handin[(X_handin["Age"] <= age+age_span) & (X_handin["Age"] >= age-age_span)].median()
    #print(medians_from_age)
    X_pid = X_pid.fillna(value=medians_from_age)
    X_handin_eng_dict["Age"].append(age)
    X_handin_eng_dict["pid"].append(pid)
    #print(X_pid)
    #pid_feats = np.array()
    for feat in all_features[3:]:
        feat_min = X_pid[feat].min()
        feat_min_name = f"{feat}_min"
        feat_max = X_pid[feat].max()
        feat_max_name = f"{feat}_max"
        feat_med = X_pid[feat].median()
        feat_med_name = f"{feat}_med"
        feat_std = X_pid[feat].std()
        feat_std_name = f"{feat}_std"
        X_handin_eng_dict[feat_min_name].append(feat_min)
        X_handin_eng_dict[feat_max_name].append(feat_max)
        X_handin_eng_dict[feat_med_name].append(feat_med)
        X_handin_eng_dict[feat_std_name].append(feat_std)
        fit_x = X_pid["Time"]
        fit_y = X_pid[feat]
        (a,b) = np.polyfit(fit_x, fit_y, 1)
        feat_a_name = f"{feat}_a"
        feat_b_name = f"{feat}_b"
        X_handin_eng_dict[feat_a_name].append(a)
        X_handin_eng_dict[feat_b_name].append(b)
print('hey')
#X_eng_dict

hey


In [7]:
X_handin_eng = pd.DataFrame.from_dict(X_handin_eng_dict)
X_handin_eng = X_handin_eng.fillna(X_handin_eng.median())

In [8]:
X_handin_eng_train = X_handin_eng.loc[:, X_handin_eng.columns != "pid"]
X_handin_eng_train.to_pickle("X_handin_eng_nopid.pkl")

In [15]:
X_handin_eng_train = X_handin_eng.loc[:, X_handin_eng.columns != "pid"]

In [22]:
X_handin_eng_train

Age  EtCO2_min  EtCO2_max  EtCO2_med  EtCO2_std  PTT_min  PTT_max  \
0       39       34.0       34.0       34.0        0.0    30.60    44.20   
1       84       30.0       30.0       30.0        0.0    32.90    32.90   
2       62       33.0       33.0       33.0        0.0    31.50    31.50   
3       71       33.0       33.0       33.0        0.0    32.60   150.00   
4       51       33.0       33.0       33.0        0.0    32.50    35.40   
...    ...        ...        ...        ...        ...      ...      ...   
12659   29       34.0       34.0       34.0        0.0    31.75    31.75   
12660   83       30.0       30.0       30.0        0.0    28.80    33.00   
12661   74       31.5       31.5       31.5        0.0    32.40    32.80   
12662   40       35.0       35.0       35.0        0.0    30.70    30.70   
12663   23       34.0       34.0       34.0        0.0    29.55    56.80   

       PTT_med       PTT_std  BUN_min  ...  TroponinI_med  TroponinI_std  \
0        30.60  4.357326e+00     12.0  ...          0.140   2.898975e-17   
1        32.90  7.421377e-15     21.0  ...          0.415   0.000000e+00   
2        31.50  0.000000e+00     16.0  ...          0.180   2.898975e-17   
3        32.60  3.368071e+01     18.0  ...          0.370   5.797951e-17   
4        32.50  8.371579e-01     15.0  ...          0.060   2.174231e-17   
...        ...           ...      ...  ...            ...            ...   
12659    31.75  0.000000e+00     12.0  ...          0.095   1.449488e-17   
12660    33.00  1.212436e+00     21.0  ...          0.340   1.159590e-16   
12661    32.80  1.154701e-01     19.0  ...          0.280   5.797951e-17   
12662    30.70  7.421377e-15      9.0  ...          0.130   3.464102e-02   
12663    29.55  7.866397e+00     12.0  ...          0.080   2.598032e+00   

       ABPs_min  ABPs_max  ABPs_med   ABPs_std  pH_min  pH_max  pH_med  \
0         119.0     144.0     120.5   8.357087    7.34    7.40    7.38   
1          82.0     180.0     121.0  24.196168    7.38    7.38    7.38   
2          90.0     127.0     106.5  10.823655    7.37    7.37    7.37   
3          94.0     159.0     129.5  15.863767    7.36    7.40    7.37   
4         111.0     138.0     126.0   8.404094    7.37    7.37    7.37   
...         ...       ...       ...        ...     ...     ...     ...   
12659     101.0     119.0     108.5   5.166911    7.37    7.37    7.37   
12660      92.0     133.0     119.0  12.099023    7.26    7.38    7.38   
12661     108.0     121.0     119.0   3.270622    7.33    7.41    7.37   
12662     100.0     152.0     125.0  17.385774    7.38    7.38    7.38   
12663     110.0     125.0     119.0   3.498918    7.37    7.37    7.37   

             pH_std  
0      1.337116e-02  
1      9.276721e-16  
2      9.276721e-16  
3      9.374369e-03  
4      9.276721e-16  
...             ...  
12659  9.276721e-16  
12660  4.923660e-02  
12661  1.729862e-02  
12662  9.276721e-16  
12663  9.276721e-16  

[12664 rows x 137 columns]

In [16]:
y_handin_pred_proba = xgb.predict_proba(X_handin_eng_train)

In [19]:
y_handin = y_handin_pred_proba[:,1]
y_handin

array([0.01045864, 0.00153811, 0.00133257, ..., 0.00409153, 0.00279249,
       0.00173799], dtype=float32)

In [20]:
pd.DataFrame(y_handin).to_csv("y_handin.csv", index=False)

In [21]:
max(y_handin)

0.03902184

In [12]:
XX = pd.DataFrame.from_dict(X_handin_eng_dict)

XX

pid  Age  EtCO2_min  EtCO2_max  EtCO2_med  EtCO2_std       EtCO2_a  \
0          0   39       34.0       34.0       34.0        0.0 -1.234403e-15   
1          3   84       30.0       30.0       30.0        0.0 -2.254118e-15   
2          5   62       33.0       33.0       33.0        0.0 -7.468214e-16   
3          7   71       33.0       33.0       33.0        0.0 -7.468214e-16   
4          9   51       33.0       33.0       33.0        0.0 -7.468214e-16   
...      ...  ...        ...        ...        ...        ...           ...   
12659  31647   29       34.0       34.0       34.0        0.0 -1.234403e-15   
12660  31649   83       30.0       30.0       30.0        0.0 -2.254118e-15   
12661  31651   74       31.5       31.5       31.5        0.0 -1.615910e-15   
12662  31652   40       35.0       35.0       35.0        0.0 -1.654362e-15   
12663  31655   23       34.0       34.0       34.0        0.0 -1.234403e-15   

       EtCO2_b  PTT_min  PTT_max  ...  ABPs_med   ABPs_std    ABPs_a  \
0         34.0    30.60    44.20  ...     120.5   8.357087 -0.500000   
1         30.0    32.90    32.90  ...     121.0  24.196168  1.804196   
2         33.0    31.50    31.50  ...     106.5  10.823655 -1.923077   
3         33.0    32.60   150.00  ...     129.5  15.863767  1.451049   
4         33.0    32.50    35.40  ...     126.0   8.404094  1.947552   
...        ...      ...      ...  ...       ...        ...       ...   
12659     34.0    31.75    31.75  ...     108.5   5.166911 -0.181818   
12660     30.0    28.80    33.00  ...     119.0  12.099023 -0.989510   
12661     31.5    32.40    32.80  ...     119.0   3.270622  0.153846   
12662     35.0    30.70    30.70  ...     125.0  17.385774 -2.646853   
12663     34.0    29.55    56.80  ...     119.0   3.498918 -0.524476   

           ABPs_b  pH_min  pH_max  pH_med        pH_std          pH_a  \
0      129.000000    7.34    7.40    7.38  1.337116e-02  1.188811e-03   
1      105.272727    7.38    7.38    7.38  9.276721e-16 -2.732902e-16   
2      121.166667    7.37    7.37    7.37  9.276721e-16 -4.989413e-16   
3      118.818182    7.36    7.40    7.37  9.374369e-03 -3.496503e-04   
4      113.424242    7.37    7.37    7.37  9.276721e-16 -4.989413e-16   
...           ...     ...     ...     ...           ...           ...   
12659  109.015152    7.37    7.37    7.37  9.276721e-16 -4.989413e-16   
12660  125.181818    7.26    7.38    7.38  4.923660e-02  3.636364e-03   
12661  117.166667    7.33    7.41    7.37  1.729862e-02 -1.643357e-03   
12662  145.787879    7.38    7.38    7.38  9.276721e-16 -2.732902e-16   
12663  122.075758    7.37    7.37    7.37  9.276721e-16 -4.989413e-16   

           pH_b  
0      7.370606  
1      7.380000  
2      7.370000  
3      7.373939  
4      7.370000  
...         ...  
12659  7.370000  
12660  7.329697  
12661  7.381515  
12662  7.380000  
12663  7.370000  

[12664 rows x 206 columns]

In [23]:
pd.DataFrame(X_handin_eng['pid']).to_csv("pids.csv", index=False, header="pid")

In [76]:
imps = xgb.get_booster().get_score(importance_type='gain')
{k: v for k, v in sorted(imps.items(), key=lambda item: item[1])}


{'Magnesium_min': 0.27521646,
 'FiO2_max': 0.307736754,
 'BaseExcess_med': 0.41181135245,
 'Phosphate_med': 0.546874106,
 'Temp_med': 0.5910143855,
 'TroponinI_std': 0.6940286135,
 'EtCO2_med': 0.7195340843333332,
 'Temp_min': 0.7656084679999999,
 'Heartrate_max': 0.782605041,
 'SaO2_min': 0.8041395743333334,
 'FiO2_min': 0.808529854,
 'AST_med': 0.816252232,
 'BUN_std': 0.8290246892,
 'HCO3_med': 0.837482393,
 'SpO2_max': 0.844277263,
 'SaO2_std': 0.8473959964,
 'BaseExcess_min': 0.851930141,
 'SpO2_med': 0.8521164354999999,
 'ABPm_max': 0.888550181888889,
 'TroponinI_min': 0.8916403758,
 'Heartrate_min': 0.8967052522857142,
 'Fibrinogen_min': 0.906343714,
 'AST_std': 0.9161482572,
 'Chloride_std': 0.9257554548,
 'PTT_max': 0.9276303046666667,
 'Platelets_min': 0.9307966148571429,
 'BaseExcess_max': 0.9496119045,
 'Glucose_std': 0.9534101335,
 'Heartrate_med': 0.972709153857143,
 'ABPd_max': 0.9807572357142856,
 'Potassium_min': 0.9832518626666666,
 'WBC_max': 0.9980353665384615,
 'Sp

In [24]:
fts = ["EtCO2", "PTT", "BUN", "Lactate", "Temp", "Hgb", "HCO3", "BaseExcess", "RRate", "Fibrinogen", "Phosphate", "WBC", "Creatinine", "PaCO2", "AST", "FiO2", "Platelets", "SaO2", "Glucose", "ABPm", "Magnesium", "Potassium", "ABPd", "Calcium", "Alkalinephos", "SpO2", "Bilirubin_direct", "Chloride", "Hct", "Heartrate", "Bilirubin_total", "TroponinI", "ABPs", "pH"]

In [26]:
fts2 = ["Age"]
for f in fts:
    fts2.append(f"{f}_min")
    fts2.append(f"{f}_max")
    fts2.append(f"{f}_med")
    fts2.append(f"{f}_std")

In [29]:
print(fts2)

['Age', 'Age_min', 'Age_max', 'Age_med', 'Age_std', 'EtCO2_min', 'EtCO2_max', 'EtCO2_med', 'EtCO2_std', 'PTT_min', 'PTT_max', 'PTT_med', 'PTT_std', 'BUN_min', 'BUN_max', 'BUN_med', 'BUN_std', 'Lactate_min', 'Lactate_max', 'Lactate_med', 'Lactate_std', 'Temp_min', 'Temp_max', 'Temp_med', 'Temp_std', 'Hgb_min', 'Hgb_max', 'Hgb_med', 'Hgb_std', 'HCO3_min', 'HCO3_max', 'HCO3_med', 'HCO3_std', 'BaseExcess_min', 'BaseExcess_max', 'BaseExcess_med', 'BaseExcess_std', 'RRate_min', 'RRate_max', 'RRate_med', 'RRate_std', 'Fibrinogen_min', 'Fibrinogen_max', 'Fibrinogen_med', 'Fibrinogen_std', 'Phosphate_min', 'Phosphate_max', 'Phosphate_med', 'Phosphate_std', 'WBC_min', 'WBC_max', 'WBC_med', 'WBC_std', 'Creatinine_min', 'Creatinine_max', 'Creatinine_med', 'Creatinine_std', 'PaCO2_min', 'PaCO2_max', 'PaCO2_med', 'PaCO2_std', 'AST_min', 'AST_max', 'AST_med', 'AST_std', 'FiO2_min', 'FiO2_max', 'FiO2_med', 'FiO2_std', 'Platelets_min', 'Platelets_max', 'Platelets_med', 'Platelets_std', 'SaO2_min', 'SaO